In [1]:
#Auxílio do Tutorial: https://matheusfacure.github.io/2017/05/12/tensorflow-essencial/

import tensorflow as tf
import gzip
import pickle
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os # para criar pastas
from sklearn.metrics import r2_score, accuracy_score

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
def process_age(df,cut_points,label_names):
    df["Age"] = df["Age"].fillna(-0.5)
    df["Age_categories"] = pd.cut(df["Age"],cut_points,labels=label_names)
    return df

cut_points = [-1,0,5,12,18,35,60,100]
label_names = ["Missing","Infant","Child","Teenager","Young Adult","Adult","Senior"]

train = process_age(train,cut_points,label_names)
test = process_age(test,cut_points,label_names)

def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    return df

train = create_dummies(train,"Pclass")
test = create_dummies(test,"Pclass")

train = create_dummies(train,"Sex")
test = create_dummies(test,"Sex")

train = create_dummies(train,"Age_categories")
test = create_dummies(test,"Age_categories")

columns = ['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male',
       'Age_categories_Missing','Age_categories_Infant',
       'Age_categories_Child', 'Age_categories_Teenager',
       'Age_categories_Young Adult', 'Age_categories_Adult',
       'Age_categories_Senior','Survived','SibSp','Parch','Fare']
columns_test = ['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male',
       'Age_categories_Missing','Age_categories_Infant',
       'Age_categories_Child', 'Age_categories_Teenager',
       'Age_categories_Young Adult', 'Age_categories_Adult',
       'Age_categories_Senior','SibSp','Parch','Fare']

df = train[columns]
df.head()

,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Age_categories_Missing,Age_categories_Infant,Age_categories_Child,Age_categories_Teenager,Age_categories_Young Adult,Age_categories_Adult,Age_categories_Senior,Survived,SibSp,Parch,Fare
0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,7.2500
1,1,0,0,1,0,0,0,0,0,0,1,0,1,1,0,71.2833
2,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,7.9250
3,1,0,0,1,0,0,0,0,0,1,0,0,1,1,0,53.1000
4,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,8.0500


In [4]:
le = preprocessing.LabelEncoder()
df_encoded = df.apply(le.fit_transform)
#list(le.classes_)
#list(le.inverse_transform([2, 2, 1]))

df_encoded.astype(float)
scaler = MinMaxScaler()
df_encoded[df_encoded.columns] = scaler.fit_transform(df_encoded[df_encoded.columns])
df_encoded.head()

X = df_encoded.drop(['Survived'], axis=1)
y = df_encoded['Survived']

X.head()

,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Age_categories_Missing,Age_categories_Infant,Age_categories_Child,Age_categories_Teenager,Age_categories_Young Adult,Age_categories_Adult,Age_categories_Senior,SibSp,Parch,Fare
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.166667,0.0,0.072874
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.166667,0.0,0.838057
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.165992
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.166667,0.0,0.765182
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.174089


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print('Formato dos dados:', X_train.shape, y_train.shape)

Formato dos dados: (596, 15) (596,)


In [6]:
# definindo constantes
lr = 1e-2 # taxa de aprendizado
n_iter = 2501 # número de iterações de treino
n_inputs = X_train.shape[1] # número de variáveis independentes
n_outputs = 1 # número de variáveis dependentes

graph = tf.Graph() # isso cria um grafo
with graph.as_default(): # isso abre o grafo para que possamos colocar operações e variáveis dentro dele.
    tf.set_random_seed(1)
    
    # adiciona as variáveis ao grafo
    W = tf.Variable(tf.truncated_normal([n_inputs, n_outputs], stddev=.1), name='Weight')
    b = tf.Variable(tf.zeros([n_outputs]), name='bias')


    ######################################
    # Monta o modelo de regressão linear #
    ######################################

    # Camadas de Inputs
    x_input = tf.placeholder(tf.float32, [None, n_inputs], name='X_input')
    y_input = tf.placeholder(tf.float32, [None, n_outputs], name='y_input')

    # Camada Linear
    y_pred = tf.add(tf.matmul(x_input, W), b, name='y_pred')

    # Camada de custo ou função objetivo
    EQM = tf.reduce_mean(tf.square(y_pred - y_input), name="EQM")

    # otimizador
    optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(EQM)

    # inicializador
    init = tf.global_variables_initializer()

    # para salvar o modelo treinado
    saver = tf.train.Saver()


In [7]:
# criamos uma pasta para salvar o modelo
if not os.path.exists('tmp'):
    os.makedirs('tmp')

# abrimos a sessão tf
with tf.Session(graph=graph) as sess:
    sess.run(init) # iniciamos as variáveis
    
    # cria um feed_dict
    feed_dict = {x_input: X_train, y_input: y_train.values.reshape(-1,1)}
    
    # realizamos as iterações de treino
    for step in range(n_iter + 1):
        
        # executa algumas operações do grafo
        _, l = sess.run([optimizer, EQM], feed_dict=feed_dict)
        
        if (step % 500) == 0:
            print('Custo na iteração %d: %.2f \r' % (step, l), end='')
            saver.save(sess, "./tmp/my_model.ckpt")


In [8]:
# novamente, abrimos a sessão tf
with tf.Session(graph=graph) as sess:
    
    # restauramos o valor das variáveis 
    saver.restore(sess, "./tmp/my_model.ckpt", )
    
    # rodamos o nó de previsão no grafo
    y_hat = sess.run(y_pred, feed_dict={x_input: X_test})
    
    print('\nR2: %.3f' % r2_score(y_pred=y_hat, y_true=y_test))
    print('\nAccuracy %.3f' % accuracy_score(y_test, y_hat.round(), normalize=True))


INFO:tensorflow:Restoring parameters from ./tmp/my_model.ckpt

R2: 0.461

Accuracy 0.820
